In [131]:
import pandas as pd

In [132]:
data = pd.read_csv('data/final_data.csv')
data['match_date'] = pd.to_datetime(data['match_date'], format="%Y-%m-%d")
data

,performance_id,match_id,match_date,player,batting_first,recent_team_scores_avg,recent_team_win_rate,recent_runs_avg,recent_wickets_avg,fantasy_score,player_label,team_name_label,opponent_team_name_label,ground_label,format_t20,format_odi,top_performer
0,7,1,2024-10-09,Jaker Ali,0,126.0,0.0,8.000000,0.000000,1.0,386,7,31,2,1,0,0
1,2,1,2024-10-09,Litton Das,0,126.0,0.0,16.666667,0.000000,14.0,539,7,31,2,1,0,0
2,6,1,2024-10-09,Mahmudullah,0,126.0,0.0,4.666667,0.000000,41.0,562,7,31,2,1,0,1
3,5,1,2024-10-09,Mehidy Hasan Miraz,0,126.0,0.0,24.000000,1.333333,16.0,607,7,31,2,1,0,0
4,11,1,2024-10-09,Mustafizur Rahman,0,126.0,0.0,0.666667,1.000000,51.0,677,7,31,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10887,13067,637,2023-12-17,Joylord Gumbie,1,166.0,0.0,5.000000,0.000000,72.0,452,93,32,31,0,1,1
10888,13071,637,2023-12-17,Sikandar Raza,1,166.0,0.0,2.000000,0.000000,37.0,951,93,32,31,0,1,1
10889,13077,637,2023-12-17,Tanaka Chivanga,1,166.0,0.0,8.000000,1.000000,6.0,986,93,32,31,0,1,0
10890,13068,637,2023-12-17,Tinashe Kamunhukamwe,1,166.0,0.0,8.000000,0.000000,1.0,1018,93,32,31,0,1,0


# split_data

In [133]:
from datetime import datetime, timedelta

split_date = '2024-10-8'
split_date_obj = datetime.strptime(split_date, '%Y-%m-%d')

prediction_date_obj = split_date_obj + timedelta(days=1)
prediction_date = prediction_date_obj.strftime('%Y-%m-%d')

print("Split Date:", split_date)
print("Prediction Date:", prediction_date)


Split Date: 2024-10-8
Prediction Date: 2024-10-09


In [134]:
train_data = data[data['match_date'] < split_date]
test_data = data[data['match_date'] >= split_date]

X_train = train_data.drop(columns=['player', 'performance_id', 'match_date', 'fantasy_score', 'top_performer'])
y_train = train_data['top_performer']

X_test = test_data.drop(columns=['player', 'performance_id', 'match_date', 'fantasy_score', 'top_performer'])
y_test = test_data['top_performer']

# model

In [135]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# model = RandomForestClassifier(n_estimators=100, random_state=42)
model = LogisticRegression()
# model = DecisionTreeClassifier(random_state=104)
# model = SVC(kernel='rbf')
# model = KNeighborsClassifier(n_neighbors=5)
# model = GaussianNB()

In [136]:
model.fit(X_train, y_train)

C:\Users\thaku\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [137]:
y_pred = model.predict(X_test)

In [138]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score

# print(f'r2_score = {r2_score(y_pred, y_test)}')
# print(f'mean_absolute_error = {mean_absolute_error(y_pred, y_test)}')
# print(f'mean_squared_error = {mean_squared_error(y_pred, y_test)}')
ac = accuracy_score(y_pred, y_test)
print(f'accuracy_score = {ac}')
print(f'team_accuracy = {ac**(22)*100}%')
print(f'correct team prediction chance: 1 in {ac**(-22)} matches')

accuracy_score = 0.45454545454545453
team_accuracy = 2.9288762467783064e-06%
correct team prediction chance: 1 in 34142787.73642198 matches


# prediction

In [139]:
data[data['match_date']>=prediction_date]['match_id'].unique()

array([1], dtype=int64)

In [141]:
def predict_match(match_id, data):
    match_data = data[data['match_id'] == match_id]
    X_prediction = match_data.drop(columns=['player', 'performance_id', 'match_date', 'fantasy_score', 'top_performer'])
    y_prediction = model.predict(X_prediction)
    
    prediction_data = match_data[['player', 'top_performer']]
    prediction_data['prediction'] = y_prediction
    
    return prediction_data

prediction_data = predict_match(1, data)
prediction_data

C:\Users\thaku\AppData\Local\Temp\ipykernel_10724\1281544943.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_data['prediction'] = y_prediction


,player,top_performer,prediction
0,Jaker Ali,0,0
1,Litton Das,0,0
2,Mahmudullah,1,0
3,Mehidy Hasan Miraz,0,0
4,Mustafizur Rahman,1,0
5,Najmul Hossain Shanto,0,0
6,Parvez Hossain Emon,0,0
7,Rishad Hossain,2,0
8,Tanzim Hasan Sakib,1,0
9,Taskin Ahmed,1,0


# prediction_1 important_but_not_important

In [141]:
def find_rank(player, rank_list):
    return (rank_list.index(player)) + 1

def predict_match(match_id, data):
    prediction_data = data[data['match_id'] == match_id]
    X_prediction = prediction_data.drop(columns=['player', 'performance_id', 'match_date', 'fantasy_score'])
    y_prediction = model.predict(X_prediction)
    
    prediction_data = prediction_data[['match_id', 'player', 'fantasy_score']]
    prediction_data['predicted_fantasy_score'] = y_prediction

    predicted_rank_list = prediction_data.sort_values(by=['predicted_fantasy_score'])['player'].tolist()
    predicted_rank_list.reverse()

    actual_rank_list = prediction_data.sort_values(by=['fantasy_score'])['player'].tolist()
    actual_rank_list.reverse()

    prediction_data['actual_rank'] = prediction_data['player'].apply(lambda x: find_rank(x, actual_rank_list))
    prediction_data['predicted_rank'] = prediction_data['player'].apply(lambda x: find_rank(x, predicted_rank_list))

    
    return prediction_data

def predict_matches(start_date, end_date, data):
    matches_data = data[data['match_date'].between(start_date, end_date)]
    match_ids = matches_data['match_id'].unique().tolist()

    prediction_data = pd.DataFrame({})
    for match_id in match_ids:
        prediction_data = pd.concat([prediction_data, predict_match(match_id, data)], ignore_index=True)
    return prediction_data

start_date = prediction_date
end_date = '2024-10-04'

prediction1_data = predict_matches(start_date, end_date, data)
prediction1_data

,match_id,player,fantasy_score,predicted_fantasy_score,actual_rank,predicted_rank
0,5,Alick Athanaze,26.0,16.24,7,22
1,5,David Miller,36.0,51.76,5,1
2,5,Jason Holder,19.0,50.05,12,3
3,5,Kadeem Alleyne,11.0,21.32,16,21
4,5,Maheesh Theekshana,30.0,32.87,6,11
...,...,...,...,...,...,...
80,8,Ryan Rickelton,91.0,53.58,3,2
81,8,Temba Bavuma,4.0,52.33,17,3
82,8,Tony de Zorzi,12.0,48.54,16,6
83,8,Tristan Stubbs,79.0,45.86,5,9


In [142]:
prediction1_data[prediction1_data['match_id']==5]

,match_id,player,fantasy_score,predicted_fantasy_score,actual_rank,predicted_rank
0,5,Alick Athanaze,26.0,16.24,7,22
1,5,David Miller,36.0,51.76,5,1
2,5,Jason Holder,19.0,50.05,12,3
3,5,Kadeem Alleyne,11.0,21.32,16,21
4,5,Maheesh Theekshana,30.0,32.87,6,11
5,5,NaveenulHaq,0.0,31.43,19,13
6,5,Obed McCoy,0.0,51.04,18,2
7,5,Quinton de Kock,15.0,24.89,14,18
8,5,Rahkeem Cornwall,19.0,25.16,13,17
9,5,Ramon Simmonds,25.0,49.70,11,4


# prediction important_but_not_important

In [133]:
def predict_match(match_id, data):
    prediction_data = data[data['match_id'] == match_id]
    X_prediction = prediction_data.drop(columns=['player', 'performance_id', 'match_date', 'fantasy_score'])
    y_prediction = model.predict(X_prediction)
    prediction_data = prediction_data[['player', 'fantasy_score']]
    prediction_data['predicted_fantasy_score'] = y_prediction
    
    predicted_sorted_df = prediction_data.sort_values(by=['predicted_fantasy_score']).iloc[-11:]
    dream_sorted_df = prediction_data.sort_values(by=['fantasy_score']).iloc[-11:]
    
    predicted_team_score = prediction_data.sort_values(by=['predicted_fantasy_score']).iloc[-11:]['fantasy_score'].sum()
    dream_team_score = prediction_data.sort_values(by=['fantasy_score']).iloc[-11:]['fantasy_score'].sum()
    
    predicted_XI = prediction_data.sort_values(by=['predicted_fantasy_score']).iloc[-11:]['player'].tolist()
    predicted_XI.sort()
    
    dream_XI = prediction_data.sort_values(by=['fantasy_score']).iloc[-11:]['player'].tolist()
    dream_XI.sort()
    
    common_players = list(filter(lambda x: x in predicted_XI, dream_XI))
    
    return predicted_sorted_df, dream_sorted_df, predicted_team_score, dream_team_score, common_players

In [134]:
data[data['match_date'] == prediction_date]['match_id'].unique()

array([7, 8], dtype=int64)

In [135]:
predicted_sorted_df, dream_sorted_df, predicted_team_score, dream_team_score, common_players = predict_match(8, data)

In [136]:
predicted_team_score

431.0

In [137]:
dream_team_score

767.0

In [138]:
print(len(common_players))
common_players

6


['Graham Hume',
 'Mark Adair',
 'Ottneil Baartman',
 'Ryan Rickelton',
 'Tristan Stubbs',
 'Wiaan Mulder']

In [139]:
predicted_sorted_df

,player,fantasy_score,predicted_fantasy_score
166,Ottneil Baartman,28.0,44.98
161,Paul Stirling,2.0,45.09
171,Tristan Stubbs,79.0,45.86
159,Harry Tector,12.0,46.25
158,Graham Hume,34.0,48.11
170,Tony de Zorzi,12.0,48.54
157,George Dockrell,21.0,48.67
160,Mark Adair,112.0,52.27
169,Temba Bavuma,4.0,52.33
168,Ryan Rickelton,91.0,53.58


In [140]:
dream_sorted_df

,player,fantasy_score,predicted_fantasy_score
166,Ottneil Baartman,28.0,44.98
158,Graham Hume,34.0,48.11
172,Wiaan Mulder,36.0,62.53
154,Andy McBrine,39.0,33.46
165,Lungi Ngidi,70.0,37.89
163,Bjorn Fortuin,78.0,34.44
171,Tristan Stubbs,79.0,45.86
155,Craig Young,87.0,35.85
168,Ryan Rickelton,91.0,53.58
160,Mark Adair,112.0,52.27
